In [1]:
import json
import requests
import pandas as pd
import importlib
from requests_oauthlib import OAuth2Session

In [2]:
token=open('token.json',mode='r')
token=token.read()
token

'{"access_token": "ya29.GlscBe7_nUkM8rlUdA3APRugfsqL6jNPWA0KADaEWDjfEg4PYs3cyU0wIepXsOAOeaK3pfwBQv_qbcD5vXMp-LwIcmQ_qHWtxXML2xmWJQPmqmx-kkw9C5Q8CC6m", "expires_in": 3600, "token_type": "Bearer", "expires_at": 1512716215.3208017, "refresh_token": "1/404Y5NzJtD0RmPCDMHRJ2QOHgM1dHW3fkogwXyJZiMQ"}'

In [3]:
token=json.loads(token)
token

{'access_token': 'ya29.GlscBe7_nUkM8rlUdA3APRugfsqL6jNPWA0KADaEWDjfEg4PYs3cyU0wIepXsOAOeaK3pfwBQv_qbcD5vXMp-LwIcmQ_qHWtxXML2xmWJQPmqmx-kkw9C5Q8CC6m',
 'expires_at': 1512716215.3208017,
 'expires_in': 3600,
 'refresh_token': '1/404Y5NzJtD0RmPCDMHRJ2QOHgM1dHW3fkogwXyJZiMQ',
 'token_type': 'Bearer'}

In [4]:
import keys
importlib.reload(keys)
keychain = keys.keychain
client_id = keychain['youtube']['client_id']
scope = keychain['youtube']['scope']
redirect = keychain['youtube']['redirect_uris'][0]
refresh_url = keychain['youtube']['token_uri']
session = OAuth2Session(client_id, scope=scope, redirect_uri=redirect)
keychain['youtube']

{'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs',
 'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
 'client_id': '126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com',
 'client_secret': 'flFfvozgUappY1MIRCPE5Cxc',
 'project_id': 'project5-187402',
 'redirect_uris': ['urn:ietf:wg:oauth:2.0:oob', 'http://localhost'],
 'scope': ['https://www.googleapis.com/auth/youtube.readonly'],
 'token_uri': 'https://accounts.google.com/o/oauth2/token'}

In [36]:
def getData(data,vid,country):
    if country != "" and "CountryCode" not in data:
        data["CountryCode"] = []
    for item in vid:
        data['videoId'].append(item['id'])
        data['channelId'].append(item['snippet']['channelId'])
        data['categoryId'].append(item['snippet']['categoryId'])
        data['channelTitle'].append(item['snippet']['channelTitle'])
        data['publishedAt'].append(item['snippet']['publishedAt'][11:13])
        if country != "":
            data['CountryCode'].append(country)
    return data

In [ ]:
def vid_by_country(token,country=""):
    D={}
    D['access_token'] = token['access_token']
    D['part'] = ['snippet,contentDetails,statistics']
    D['maxResults'] = '50'
    D['chart'] ='mostPopular'
    D['PageToken'] = 'CAUQAA'
    D["RegionalCode"] = country
    url ='https://www.googleapis.com/youtube/v3/videos'
    popular_vid = requests.get(url, params = D)
    popular_vid = popular_vid.json()
    
    page = []
    data = {'videoId':[], 'channelId':[],'categoryId':[] ,'channelTitle':[], 'publishedAt':[]}
    for i in range(10):
        if 'nextPageToken' not in popular_vid:
            next_page = 'CAUQAA'
        else:
            next_page = popular_vid['nextPageToken']
        page.append(next_page)
        D['PageToken'] = next_page
        popular_vid = requests.get(url, params = D)
        popular_vid = popular_vid.json()
        getData(data,popular_vid['items'],country)
        
    df= pd.DataFrame(data)
    return df

df_global = vid_by_country(token)
df_global.to_csv('df_global.csv')

df_US = vid_by_country(token,"US")
df_US.to_csv('df_US.csv')

df_France = vid_by_country(token,"FR")
df_France.to_csv('df_France.csv')

df_China = vid_by_country(token,"CN")
df_China.to_csv('df_China.csv')

df_Spain = vid_by_country(token,"ES")
df_Spain.to_csv('df_Spain.csv')

df_global[:10]

In [40]:
url = 'https://www.googleapis.com/youtube/v3/videoCategories'
C = {}
C['access_token'] = token['access_token']
C['part'] = 'snippet'
C['regionCode'] = 'US'
category_vid = requests.get(url, params = C)
category_vid = category_vid.json()

In [41]:
data_category = {'id':[],'title':[]}
def categoryTable(data,json):
    for item in json:
        data['id'].append(item['id'])
        data['title'].append(item['snippet']['title'])
categoryTable(data_category,category_vid['items'])
df1 = pd.DataFrame(data_category)
df1[:10]

,id,title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [42]:
result = pd.merge(df_global, df1, left_on='categoryId', right_on='id')
result.to_csv('result.csv')

In [7]:
token = session.refresh_token(refresh_url, 
                             client_id=keychain['youtube']['client_id'],
                             client_secret=keychain['youtube']['client_secret'],
                             refresh_token=token['refresh_token'])